In [1]:
## IMPORTS

import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForTokenClassification, AdamW

KeyboardInterrupt: 

In [ ]:
data = [
    {"text": "2022 óleo sobre tela", "label": "óleo sobre tela"},
    {"text": "Obra faz parte da série Enquanto isso, lá foraRealizada sobre painel telado com carvão e cera de abelha Emoldurada em madeira preta", "label": "carvão e cera de abelha sobre painel telado"},
    {"text": "2022 Óleo sobre tela alo eu sou legal", "label": "óleo sobre tela"},
    {"text": "Mulheres do Bordel, déc. 1950. Aquarela e nanquim sobre cartão.", "label": "aquarela e nanquim sobre cartão"}
]

def convert_to_iob_format(data):
    iob_data = []
    for entry in data:
        text = entry['text']
        label = entry['label']
        
        tokens = text.split()
        label_tokens = label.split()
        
        iob_labels = ['O'] * len(tokens)
        
        # Find the start and end positions of the label in the text
        start_pos = text.find(label_tokens[0])
        end_pos = start_pos + len(label_tokens[0])
        
        # Mark the tokens within the label as 'B-LABEL' and 'I-LABEL'
        iob_labels[start_pos:end_pos] = ['B-LABEL'] + ['I-LABEL'] * (len(label_tokens) - 1)
        
        # Create a new dictionary with IOB-formatted labels
        iob_entry = {
            'tokens': tokens,
            'labels': iob_labels
        }
        
        iob_data.append(iob_entry)
    
    return iob_data

iob_data = convert_to_iob_format(data)
print(iob_data)


[{'tokens': ['2022', 'óleo', 'sobre', 'tela'], 'labels': ['O', 'O', 'O', 'O', 'B-LABEL', 'I-LABEL', 'I-LABEL']}, {'tokens': ['Obra', 'faz', 'parte', 'da', 'série', 'Enquanto', 'isso,', 'lá', 'foraRealizada', 'sobre', 'painel', 'telado', 'com', 'carvão', 'e', 'cera', 'de', 'abelhaEmoldurada', 'em', 'madeira', 'preta'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL']}, {'tokens': ['2022', 'Óleo', 'sobre', 'tela', 'alo', 'eu', 'sou', 'legal'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LABEL', 'I-LABEL', 'I-LABEL', 'O']}, {'tokens': ['Mulheres', 'do', 'Bordel,', 'déc.', '1950.', 'Aquarela', 'e', 'nanquim', 'sobre', 'cartão.'], 'labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL', 'I-LABEL', 'O']}]


In [ ]:
# Define the labeled dataset and preprocessing functions
# ...

# Create a custom dataset for PyTorch DataLoader
class ArtworkDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long),
        }

# Define the BERT-based NER model
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels)

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Define your DataLoader with the custom dataset
train_dataset = ArtworkDataset(train_texts, train_labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        
        optimizer.zero_grad()
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluate the model
# ...

# Test the model
# ...



Materials (text1): []
Mediums (text1): []
Materials (text2): []
Mediums (text2): []
